In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
from api.parsers_api import headers, HTTP_error, parse_GameSpot
from tqdm import tqdm

In [185]:
url_0 = "https://www.metacritic.com/game/nintendo-64/tony-hawks-pro-skater-2/details"

In [195]:
first_link = url_0
r = requests.get( first_link, headers=headers)
soup = bs(r.text, "html.parser")
head = soup.find(class_="module product_data")
desc = head.find(class_="data").text

meta_score = head.find(itemprop="ratingValue").text
meta_count = head.find(class_="count").a.span.text.strip()

user_score = head.find(class_="details side_details").find(class_="metascore_anchor").div.text
user_count = head.find(class_="details side_details").find(class_="count").a.text[:-8]

data = soup.find(class_="summary_detail release_data").find(class_="data").text
platforms = []

p_span_platform = soup.find(class_="summary_detail product_platforms")

if  p_span_platform:
    span_platforms = p_span_platform.find(class_="data").find_all("a")
    for a_platform in span_platforms:
        platforms.append(a_platform.text)

#for platform of the page
platforms.append(soup.find(class_ = "product_title").find(class_ = "platform").text.strip())

table = soup.find(class_ = "module product_data").find_next_sibling()
rating = None
site = None
developer = None
genre = None
if table:
    continue


list_tr = table.find_all("tr")
for row in list_tr:
    # print(row.th.text[:-1], row.td.text)
    if row.th.text[:-1] == "Rating":
        rating = row.td.text
    if row.th.text[:-1] == "Official Site":
        site = row.a.text
    if row.th.text[:-1] == "Developer":
        developer = row.td.text
    if row.th.text[:-1] == "Genre(s)":
        genre = row.td.text.strip().split()

print(meta_score,meta_count)
print(user_score,user_count)
print(data, platforms)
print(rating,site, developer, genre)

84 10
8.6 54
Aug 21, 2001 ['iPhone/iPad', 'PC', 'Nintendo 64']
E None Edge of Reality ['Sports,', 'Alternative,', 'Skateboarding']


In [200]:
with open("/Users/fedor/Desktop/meta_project/data/доп/список_страниц_игр.json", "r") as f:
    URLS = json.load(f)

In [246]:
actual_position = 48 + 104+8+1+376+31+6+2055+519

In [247]:
count = 0
for name in tqdm(list(URLS.keys())[actual_position:]):

    link =  "https://www.metacritic.com" + URLS[name][:-15] + "/details"
    # print(link)
    r = requests.get( link, headers=headers)
    if r.status_code != requests.codes.ok:
        continue
    soup = bs(r.text, "html.parser")
    head = soup.find(class_="module product_data")

    meta_score = None
    if head.find(itemprop="ratingValue"):
        meta_score = head.find(itemprop="ratingValue").text

    meta_count = None
    if head.find(class_="count").a.span:
        meta_count = head.find(class_="count").a.span.text.strip()

    user_score = None
    if head.find(class_="details side_details").find(class_="metascore_anchor"):
        user_score = head.find(class_="details side_details").find(class_="metascore_anchor").div.text

    user_count = None
    if head.find(class_="details side_details").find(class_="count").a:
        user_count = head.find(class_="details side_details").find(class_="count").a.text[:-8]

    data = soup.find(class_="summary_detail release_data").find(class_="data").text
    platforms = []

    p_span_platform = soup.find(class_="summary_detail product_platforms")

    if  p_span_platform:
        span_platforms = p_span_platform.find(class_="data").find_all("a")
        for a_platform in span_platforms:
            platforms.append(a_platform.text)

    #for platform of the page
    platforms.append(soup.find(class_ = "product_title").find(class_ = "platform").text.strip())

    table = soup.find(class_ = "module product_data").find_next_sibling()

    rating = None
    site = None
    developer = None
    genre = None

    list_tr = table.find_all("tr")
    for row in list_tr:
        if row.th.text[:-1] == "Rating":
            rating = row.td.text
        if row.th.text[:-1] == "Official Site":
            site = row.a.text
        if row.th.text[:-1] == "Developer":
            developer = row.td.text
        if row.th.text[:-1] == "Genre(s)":
            genre = row.td.text.strip().split()

    # print(name)
    # print(link)
    # print(meta_score,meta_count)
    # print(user_score,user_count)
    # print(data, platforms)
    # print(rating,site, developer, genre)
    with open("/Users/fedor/Desktop/meta_project/data/описания/"+str(count+actual_position)+".json", "w") as f:
        json.dump({
            "name": name,
            "URL" : link,
            "meta_score" : meta_score,
            "meta_count":meta_count,
            "user_score":user_score,
            "user_count":user_count,
            "data":data,
            "platforms":platforms,
            "rating":rating,
            "official_site":site,
            "developer":developer,
            "genre":genre,
        }, f)
        count += 1
    # break

100%|██████████| 9976/9976 [3:09:46<00:00,  1.14s/it]  


In [215]:
print(URLS[list(URLS.keys())[actual_position+1]])

/game/pc/madden-nfl-2003/critic-reviews


In [248]:
len(URLS)

13124